In [39]:
# Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score, hamming_loss, f1_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

provided_weights_genetic_disorder = {
    'Mitochondrial genetic inheritance disorders': 0.48794813542417026,
    'Single-gene inheritance diseases': 0.6160580705934504,
    'Multifactorial genetic inheritance disorders': 0.8959937939823793
}

provided_weights_subclass_disorder = {
    'Leigh syndrome': 0.740510888236272,
    'Mitochondrial myopathy': 0.7799634288247355,
    'Cystic fibrosis': 0.8257328087770821,
    'Tay-Sachs': 0.8583698121571453,
    'Diabetes': 0.9084058292236937,
    'Hemochromatosis': 0.9319554496592232,
    "Leber's hereditary optic neuropathy": 0.9674738183631628,
    "Alzheimer's": 0.9926303540754696,
    'Cancer': 0.9949576106832161
}


In [27]:
# Load dataset
dataset= pd.read_csv('dataset.csv')
#print(dataset.columns)

# Replace 'Unknown' with NaN
dataset.replace('Unknown', np.nan, inplace=True)

# Eliminate samples with
dataset.dropna(inplace=True)


# Features to use
features = ['Patient Age', "Mother's age", "Father's age", 'No. of previous abortion',
    'White Blood cell count (thousand per microliter)']
targets = ['Genetic Disorder', 'Disorder Subclass']

# Select only the desired columns
selected_columns = features + targets
dataset = dataset[selected_columns]



# Encode categorical variables
label_encoder = LabelEncoder()

# Create a copy for encoding
dataset_encoded = dataset.copy()

quantitative_with_unknowns_or_ordered_columns = [
    'Patient Age', "Mother's age", "Father's age", 'No. of previous abortion',
    'White Blood cell count (thousand per microliter)']



# Encode target variables 'Genetic Disorder' and 'Disorder Subclass'
# Create mapping
genetic_disorder_mapping = {label: i for i, label in enumerate(dataset_encoded['Genetic Disorder'].unique()) if pd.notna(label)}
disorder_subclass_mapping = {label: i for i, label in enumerate(dataset_encoded['Disorder Subclass'].unique())}

# Replace each original value with its corresponding encoded value as per the mapping
dataset_encoded['Genetic Disorder'] = dataset_encoded['Genetic Disorder'].map(genetic_disorder_mapping)
dataset_encoded['Disorder Subclass'] = dataset_encoded['Disorder Subclass'].map(disorder_subclass_mapping)


# Correspond weights to classes
# Get values mapped
encoded_values_genetic_disorder = dataset_encoded['Genetic Disorder'].unique()
encoded_values_disorder_subclass = dataset_encoded['Disorder Subclass'].unique()

# Inverse mappings to get back the original names
inverse_genetic_disorder_mapping = {i: label for label, i in genetic_disorder_mapping.items()}
inverse_disorder_subclass_mapping = {i: label for label, i in disorder_subclass_mapping.items()}

# Map encoded values back to original names
names_genetic_disorder = [inverse_genetic_disorder_mapping[i] for i in encoded_values_genetic_disorder]
names_disorder_subclass = [inverse_disorder_subclass_mapping[i] for i in encoded_values_disorder_subclass]

# Associate weights with encoded values
weights_genetic_disorder = {encoded_value: provided_weights_genetic_disorder[name] for encoded_value, name in zip(encoded_values_genetic_disorder, names_genetic_disorder)}
weights_disorder_subclass = {encoded_value: provided_weights_subclass_disorder[name] for encoded_value, name in zip(encoded_values_disorder_subclass, names_disorder_subclass)}

# Combine both class weights into a dictionary
class_weights = {'Genetic Disorder': weights_genetic_disorder, 'Disorder Subclass': weights_disorder_subclass}


# Allocate features and labels
X = dataset_encoded.drop(columns=['Genetic Disorder', 'Disorder Subclass'])
y = dataset_encoded[['Genetic Disorder', 'Disorder Subclass']]

In [28]:
# Select and train model of classifier

# Best Hyperparameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 10, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10, 'estimator__splitter': 'random'}
# Best Mean Accuracy: 0.2690801625415589
'''
dt = MultiOutputClassifier(tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 1, min_samples_split= 10, splitter = 'random', random_state = 1))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

cv_scores = cross_val_score(dt, X_train, y_train, cv=kf)

print("Cross-validated scores:", cv_scores)
print("Mean cross-validated score:", cv_scores.mean())'''


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=1)


# Define the class weights for both outputs
class_weights_genetic_disorder = {0: 0.48794813542417026, 1: 0.8959937939823793, 2: 0.6160580705934504}
class_weights_disorder_subclass = {0: 0.740510888236272, 1: 0.9084058292236937, 2: 0.7799634288247355, 3: 0.8257328087770821, 4: 0.8583698121571453, 5: 0.9674738183631628, 6: 0.9319554496592232, 7: 0.9926303540754696, 8: 0.9949576106832161}

# DecisionTreeClassifier with specified parameters and class weights
dt_classifier_genetic_disorder = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=10,
    splitter='random',
    random_state=1,
    class_weight=class_weights_genetic_disorder
)

dt_classifier_disorder_subclass = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=10,
    splitter='random',
    random_state=1,
    class_weight=class_weights_disorder_subclass
)

# Create a MultiOutputClassifier
dt = MultiOutputClassifier(estimator=DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=10,
    splitter='random',
    random_state=1
), n_jobs=-1)

# Fit the model with sample weights
dt.fit(X_train, y_train)

# Predict using the trained model
y_pred = dt.predict(X_test)
proba_pred = dt.predict_proba(X_test)

In [29]:
# Get 12 targets to evaluate model
'''y_test_columns = y_test.columns[-12:] 
y_pred_columns = y_pred[:, -12:]

balanced_acc_scores_ = {}

for i, column in enumerate(y_test_columns):
    y_test_ = y_test[column]
    y_pred_ = y_pred_columns[:, i]
    
    balanced_acc_scores_[column] = balanced_accuracy_score(y_test_, y_pred_)'''
# y_test_new is DataFrames
y_test_columns = y_test.columns[-2:]
# Convert y_pred_new to a DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=y_test_columns)

balanced_acc_scores = {}

for column in y_test_columns:
    y_test_column = y_test[column]
    y_pred_column = y_pred_df[column]
    
    balanced_acc_scores[column] = balanced_accuracy_score(y_test_column, y_pred_column)


In [30]:
# Generate classification report for each output separately

y_test_array = y_test.to_numpy()

hamming_loss_genetic_disorder = hamming_loss(y_test_array[:, 0], y_pred[:, 0])
hamming_loss_disorder_subclass = hamming_loss(y_test_array[:, 1], y_pred[:, 1])

f1_score_genetic_disorder = f1_score(y_test_array[:, 0], y_pred[:, 0], average='weighted')
f1_score_disorder_subclass = f1_score(y_test_array[:, 1], y_pred[:, 1], average='weighted')

recall_genetic_disorder = recall_score(y_test_array[:, 0], y_pred[:, 0], average='weighted')
recall_disorder_subclass = recall_score(y_test_array[:, 1], y_pred[:, 1], average='weighted')

balanced_accuracy_genetic_disorder = balanced_accuracy_score(y_test_array[:, 0], y_pred[:, 0])
balanced_accuracy_disorder_subclass = balanced_accuracy_score(y_test_array[:, 1], y_pred[:, 1])

# Calculate Hamming loss for each output separately

print("Classification for Genetic Disorder:")

print("Hamming Loss for Genetic Disorder:", hamming_loss_genetic_disorder)
print("F1 Score for Genetic Disorder:", f1_score_genetic_disorder)
print("Recall for Genetic Disorder:", recall_genetic_disorder)
print("Balanced Accuracy for Genetic Disorder:", balanced_accuracy_genetic_disorder)

print("\nClassification for Disorder Subclass:")

print("Hamming Loss for Disorder Subclass:", hamming_loss_disorder_subclass)
print("F1 Score for Disorder Subclass:", f1_score_disorder_subclass)
print("Recall for Disorder Subclass:", recall_disorder_subclass)
print("Balanced Accuracy for Disorder Subclass:", balanced_accuracy_disorder_subclass)

Classification for Genetic Disorder:
Hamming Loss for Genetic Disorder: 0.5657370517928287
F1 Score for Genetic Disorder: 0.41103190144989044
Recall for Genetic Disorder: 0.4342629482071713
Balanced Accuracy for Genetic Disorder: 0.29760770615010057

Classification for Disorder Subclass:
Hamming Loss for Disorder Subclass: 0.7729083665338645
F1 Score for Disorder Subclass: 0.20727974332828783
Recall for Disorder Subclass: 0.22709163346613545
Balanced Accuracy for Disorder Subclass: 0.11098237274707863


# Use dataset without Cancer and Alzeimer

In [31]:
# Assuming your DataFrame is named dataset
filtered_dataset = dataset[(dataset['Disorder Subclass'] != "Cancer") & (dataset['Disorder Subclass'] != "Alzheimer's")]

# Update your X and y with the filtered dataset
X_filtered = filtered_dataset.drop(columns=['Genetic Disorder', 'Disorder Subclass'])
y_filtered = filtered_dataset[['Genetic Disorder', 'Disorder Subclass']]


X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_filtered, y_filtered, train_size=0.75,  random_state=1) # training as 75%

In [37]:
# Select and train model of classifier

# Best Hyperparameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 10, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10, 'estimator__splitter': 'random'}
# Best Mean Accuracy: 0.2690801625415589
'''
dt = MultiOutputClassifier(tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 1, min_samples_split= 10, splitter = 'random', random_state = 1))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

cv_scores = cross_val_score(dt, X_train, y_train, cv=kf)

print("Cross-validated scores:", cv_scores)
print("Mean cross-validated score:", cv_scores.mean())'''


# Define the class weights for both outputs
class_weights_genetic_disorder = {0: 0.48794813542417026, 1: 0.8959937939823793, 2: 0.6160580705934504}
class_weights_disorder_subclass = {0: 0.740510888236272, 1: 0.9084058292236937, 2: 0.7799634288247355, 3: 0.8257328087770821, 4: 0.8583698121571453, 5: 0.9674738183631628, 6: 0.9319554496592232, 7: 0.9926303540754696, 8: 0.9949576106832161}

# DecisionTreeClassifier with specified parameters and class weights
dt_classifier_genetic_disorder_new = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=10,
    splitter='random',
    random_state=1,
    class_weight=class_weights_genetic_disorder
)

dt_classifier_disorder_subclass_new = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=10,
    splitter='random',
    random_state=1,
    class_weight=class_weights_disorder_subclass
)

# Create a MultiOutputClassifier
dt_new = MultiOutputClassifier(estimator=DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=10,
    splitter='random',
    random_state=1
), n_jobs=-1)

# Fit the model with sample weights
dt_new.fit(X_train_new, y_train_new)

# Predict using the trained model
y_pred_new = dt_new.predict(X_test_new)
proba_pred_new = dt_new.predict_proba(X_test_new) 


In [38]:
# Get 12 targets to evaluate model
'''y_test_columns = y_test.columns[-12:] 
y_pred_columns = y_pred[:, -12:]

balanced_acc_scores_ = {}

for i, column in enumerate(y_test_columns):
    y_test_ = y_test[column]
    y_pred_ = y_pred_columns[:, i]
    
    balanced_acc_scores_[column] = balanced_accuracy_score(y_test_, y_pred_)'''
# y_test_new is DataFrames
y_test_columns_new = y_test_new.columns[-2:]
# Convert y_pred_new to a DataFrame
y_pred_df_new = pd.DataFrame(y_pred_new, columns=y_test_columns_new)

balanced_acc_scores_new = {}

for column in y_test_columns_new:
    y_test_column_new = y_test_new[column]
    y_pred_column_new = y_pred_df_new[column]
    
    balanced_acc_scores_new[column] = balanced_accuracy_score(y_test_column_new, y_pred_column_new)


In [40]:
# Generate classification report for each output separately

y_test_array_new = y_test_new.to_numpy()

hamming_loss_genetic_disorder = hamming_loss(y_test_array_new[:, 0], y_pred_new[:, 0])
hamming_loss_disorder_subclass = hamming_loss(y_test_array_new[:, 1], y_pred_new[:, 1])

f1_score_genetic_disorder = f1_score(y_test_array_new[:, 0], y_pred_new[:, 0], average='weighted')
f1_score_disorder_subclass = f1_score(y_test_array_new[:, 1], y_pred_new[:, 1], average='weighted')

recall_genetic_disorder = recall_score(y_test_array_new[:, 0], y_pred_new[:, 0], average='weighted')
recall_disorder_subclass = recall_score(y_test_array_new[:, 1], y_pred_new[:, 1], average='weighted')

balanced_accuracy_genetic_disorder = balanced_accuracy_score(y_test_array_new[:, 0], y_pred_new[:, 0])
balanced_accuracy_disorder_subclass = balanced_accuracy_score(y_test_array_new[:, 1], y_pred_new[:, 1])

# Calculate Hamming loss for each output separately

print("Classification for Genetic Disorder:")

print("Hamming Loss for Genetic Disorder:", hamming_loss_genetic_disorder)
print("F1 Score for Genetic Disorder:", f1_score_genetic_disorder)
print("Recall for Genetic Disorder:", recall_genetic_disorder)
print("Balanced Accuracy for Genetic Disorder:", balanced_accuracy_genetic_disorder)

print("\nClassification for Disorder Subclass:")

print("Hamming Loss for Disorder Subclass:", hamming_loss_disorder_subclass)
print("F1 Score for Disorder Subclass:", f1_score_disorder_subclass)
print("Recall for Disorder Subclass:", recall_disorder_subclass)
print("Balanced Accuracy for Disorder Subclass:", balanced_accuracy_disorder_subclass)



Classification for Genetic Disorder:
Hamming Loss for Genetic Disorder: 0.5662650602409639
F1 Score for Genetic Disorder: 0.4129901638332833
Recall for Genetic Disorder: 0.43373493975903615
Balanced Accuracy for Genetic Disorder: 0.3148387096774194

Classification for Disorder Subclass:
Hamming Loss for Disorder Subclass: 0.8032128514056225
F1 Score for Disorder Subclass: 0.1751466384968828
Recall for Disorder Subclass: 0.19678714859437751
Balanced Accuracy for Disorder Subclass: 0.14622150614631815


In [20]:
# Metrics and information for each output column
print('\nClassification Report:')
print(classification_report(y_test, y_pred))
for column, score in balanced_acc_scores.items():
    print(f'Balanced Accuracy for {column}: {score}')

print('\n-------------------------REMOVING UNKNOWNS ---------------------------')

print('\nClassification Report:')
print(classification_report(y_test_new, y_pred_new))

for column, score in balanced_acc_scores_new.items():
    print(f'Balanced Accuracy for {column}: {score}')



Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.58      0.55      2343
           1       0.20      0.00      0.01       471
           2       0.37      0.13      0.20      1698
           3       0.00      0.00      0.00        32
           4       0.00      0.00      0.00        22
           5       0.07      0.00      0.01       745
           6       0.22      0.00      0.01       417
           7       0.00      0.00      0.00       315
           8       0.00      0.00      0.00       141
           9       0.25      0.01      0.03      1186
          10       0.31      0.01      0.03      1016
          11       0.15      0.01      0.01       638

   micro avg       0.47      0.18      0.26      9024
   macro avg       0.17      0.06      0.07      9024
weighted avg       0.31      0.18      0.19      9024
 samples avg       0.35      0.18      0.24      9024

Balanced Accuracy for Genetic Disorder_Mitochondrial ge

c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\